# Data Cleaning and Orginization

The data I exported from fitbit website contains folders for all the metrics, some metrics are recorded in different ways (ie. some are summarized by day, some are summarized by month, others are recorded by specific interval ). Therefore, I need to organize them uniformly to get a overall dataframe.

### 1. Merge Data Into Single Dataframe with Corresponding Dates For Each Variable
### 2. Rename Columns For Each Variable

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

'Variable 1 : Active Zone Minutes'
# keep only 'date_time' and 'total_minutes' columns
AZM_df = pd.read_csv('ActiveZoneMinutes.csv')
AZM_df = AZM_df.drop('heart_zone_id', axis=1)

# convert format
AZM_df['date_time'] = pd.to_datetime(AZM_df['date_time'])
AZM_df['total_minutes'] = pd.to_numeric(AZM_df['total_minutes'], errors='coerce')

# group by date and sum total minutes
AZM_grouped = AZM_df.groupby(AZM_df['date_time'].dt.date)
sum_total_minutes = AZM_grouped['total_minutes'].sum()
AZM_df = sum_total_minutes.reset_index()

# rename columns
new_AZM = {'date_time' : 'date', 'total_minutes' : 'AZM_minutes'}
AZM_df = AZM_df.rename(columns=new_AZM)

print(AZM_df.head(5))
print(AZM_df.shape)

         date  AZM_minutes
0  2023-06-27         77.0
1  2023-06-28         32.0
2  2023-06-29        157.0
3  2023-06-30         34.0
4  2023-07-01          1.0
(164, 2)


In [4]:
'Variable 2 : Daily Heart Rate Variability'
DHR_df = pd.read_csv('DailyHeartRateVariability.csv')
# keep 'timestamp', 'rmssd', 'nremhr', 'entropy'
DHR_columns = ['timestamp', 'rmssd', 'nremhr', 'entropy']
DHR_df = DHR_df[DHR_columns]

# convert format
DHR_df['timestamp'] = pd.to_datetime(DHR_df['timestamp']).dt.date
numeric_columns = ['rmssd', 'nremhr', 'entropy']
DHR_df[numeric_columns] = DHR_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# take mean of each variable
DHR_df = DHR_df.groupby(DHR_df['timestamp']).mean().reset_index()

# rename column
new_DHR = {'timestamp' : 'date'}
DHR_df = DHR_df.rename(columns=new_DHR)

print(DHR_df.head(5))
print(DHR_df.shape)

         date      rmssd    nremhr      entropy
0  2023-06-28  56.078825  0.972186   808.054515
1  2023-06-29  67.893931  0.969713  1106.613195
2  2023-06-30  63.092575  0.974014   930.920795
3  2023-07-01  87.917760  0.967302  1320.889031
4  2023-07-02  60.617967  0.971125   950.854017
(162, 4)


/var/folders/tp/6kwhdvr504795tj0h4_byy2c0000gn/T/ipykernel_29271/2574532058.py:2: DtypeWarning: Columns (1,2,4,5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  DHR_df = pd.read_csv('DailyHeartRateVariability.csv')


In [5]:
'variable 3 : sleep score'
# since restlessness column depend on day performance and there are days I was not wearing
# I decide to drop this column
# https://www.zdnet.com/article/fitbits-sleep-score-beta/

sleep_df = pd.read_csv('sleep_score.csv')
sleep_df = sleep_df[['timestamp', 'overall_score', 'deep_sleep_in_minutes', 'resting_heart_rate']]

# convert format
sleep_df['timestamp'] = pd.to_datetime(sleep_df['timestamp']).dt.date # dt.date to keep date only, disregard time
numeric_columns3 = ['overall_score', 'deep_sleep_in_minutes', 'resting_heart_rate']
sleep_df[numeric_columns3] = sleep_df[numeric_columns3].apply(pd.to_numeric, errors='coerce')
sleep_df = sleep_df.sort_values(by='timestamp') # sort in ascending order

# rename columns
new_sleep = {'timestamp' : 'date', 'overall_score' : 'sleep_score', 'deep_sleep_in_minutes' : 'deep_sleep_min'}
sleep_df = sleep_df.rename(columns=new_sleep)
sleep_df = sleep_df.reset_index(drop=True)

print(sleep_df.head(5))
print(sleep_df.shape)

         date  sleep_score  deep_sleep_min  resting_heart_rate
0  2023-06-29           68              96                  58
1  2023-06-30           65              65                  57
2  2023-07-01           85             106                  57
3  2023-07-02           84              90                  56
4  2023-07-03           80              78                  56
(161, 4)


In [6]:
'variable 4 : stress score'
# I am unsure about how stress_score is calculated but I will keep this variable for now
# only keep columns 'date', 'stress_score'

stress_df = pd.read_csv('stress_score.csv')
stress_df = stress_df[['DATE', 'STRESS_SCORE']]

# convert format
stress_df['DATE'] = pd.to_datetime(stress_df['DATE']).dt.date
new_stress = {'DATE' : 'date', 'STRESS_SCORE' : 'stress_score'}
stress_df = stress_df.rename(columns=new_stress)

# remove rows with stress_score == 0
stress_df = stress_df[stress_df['stress_score'] != 0]

print(stress_df.head(5))
print(stress_df.shape)

         date  stress_score
1  2023-06-29            77
2  2023-06-30            80
3  2023-07-01            86
4  2023-07-02            79
5  2023-07-03            82
(158, 2)


In [7]:
'variable 5 : oxygen saturation'

o2_df = pd.read_csv('o2.csv')

# convert format
o2_df['date'] = pd.to_datetime(o2_df['date']).dt.date
num_o2 = ['o2_avg', 'o2_lower_bound', 'o2_upper_bound']
o2_df[num_o2] = o2_df[num_o2].apply(pd.to_numeric, errors='coerce')

# take mean of each variable
o2_df = o2_df.groupby(o2_df['date']).mean().reset_index()

print(o2_df.head(5))
print(o2_df.shape)

         date     o2_avg  o2_lower_bound  o2_upper_bound
0  2023-06-28  87.149551           90.05            99.7
1  2023-06-29  84.797266           70.70            98.8
2  2023-06-30  83.358629           93.05            98.4
3  2023-07-01  84.843333           86.35            98.6
4  2023-07-02  84.867290           86.75            98.2
(165, 4)


### 3. Merge ALL Data Into Single Dataframe 

In [12]:
'merge all df into one by date'
# fitbit_df = pd.concat([AZM_df, DHR_df, sleep_df, stress_df, o2_df], axis=1)

from functools import reduce

dataframes = [AZM_df, DHR_df, sleep_df, stress_df, o2_df]
fitbit_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dataframes)

print(fitbit_df.head(5))
print(fitbit_df.tail(5))

         date  AZM_minutes      rmssd    nremhr      entropy  sleep_score  \
0  2023-06-27         77.0        NaN       NaN          NaN          NaN   
1  2023-06-28         32.0  56.078825  0.972186   808.054515          NaN   
2  2023-06-29        157.0  67.893931  0.969713  1106.613195         68.0   
3  2023-06-30         34.0  63.092575  0.974014   930.920795         65.0   
4  2023-07-01          1.0  87.917760  0.967302  1320.889031         85.0   

   deep_sleep_min  resting_heart_rate  stress_score     o2_avg  \
0             NaN                 NaN           NaN        NaN   
1             NaN                 NaN           NaN  87.149551   
2            96.0                58.0          77.0  84.797266   
3            65.0                57.0          80.0  83.358629   
4           106.0                57.0          86.0  84.843333   

   o2_lower_bound  o2_upper_bound  
0             NaN             NaN  
1           90.05            99.7  
2           70.70            98.

### 4. Drop NA values

There is not significant number of rows we removed

In [13]:
fitbit_df = fitbit_df.dropna()
print(fitbit_df.shape)

(151, 12)


### 5. Save fitbit_df into csv format

In [15]:
fitbit_df.to_csv('fitbit_data.csv', index=False)

We finally obtained the complete data that is ready to be analyzed!